# Youtube Preprocessing

In [165]:
import pandas as pd
from datetime import datetime, timedelta
import re


In [166]:
df = pd.read_excel('../Scraping_Output/facebook_data.csv')
df

,user_id,platform,post_id,date,likes,comments,shares,post_text,post_origin_text,date_scraped,views,followers,country,content_type,sentiment_score
0,aljazeerachannel,Facebook,27c8d322-f361-48cc-bf5b-a886062806f0,NaN,356,64,12,Breaking | Reuters on diplomats: #Washington a...,Al Jazeera Channel - قناة الجزيرة\n n\nd\no\ne...,2025-03-09 22:18:20,NaN,30M followers,NaN,image,NaN


In [167]:



# Remove empty rows where post_origin_text is empty
df = df.dropna(subset=['post_origin_text'])

# Reset the index after dropping rows
df = df.reset_index(drop=True)

df.head(2)

,user_id,platform,post_id,date,likes,comments,shares,post_text,post_origin_text,date_scraped,views,followers,country,content_type,sentiment_score
0,aljazeerachannel,Facebook,27c8d322-f361-48cc-bf5b-a886062806f0,NaN,356,64,12,Breaking | Reuters on diplomats: #Washington a...,Al Jazeera Channel - قناة الجزيرة\n n\nd\no\ne...,2025-03-09 22:18:20,NaN,30M followers,NaN,image,NaN


In [168]:
unique_dates = df['date'].unique()
print(unique_dates)

[nan]


In [169]:
# Function to extract time (e.g., '6h')
def extract_time(post_text):
    # Matching time formats like '6h', '5d', '2m', '1y'
    match = re.search(r'(\d+)(h|d|m|y)', post_text)
    return match.group(0) if match else None

# Applying the function to the 'post_text' column and creating a new column 'time'
df['date_difference'] = df['post_origin_text'].apply(extract_time)


# Remove empty rows where post_origin_text is empty
df = df.dropna(subset=['date_difference'])

# Reset the index after dropping rows
df = df.reset_index(drop=True)


df.head(2)

,user_id,platform,post_id,date,likes,comments,shares,post_text,post_origin_text,date_scraped,views,followers,country,content_type,sentiment_score,date_difference
0,aljazeerachannel,Facebook,27c8d322-f361-48cc-bf5b-a886062806f0,NaN,356,64,12,Breaking | Reuters on diplomats: #Washington a...,Al Jazeera Channel - قناة الجزيرة\n n\nd\no\ne...,2025-03-09 22:18:20,NaN,30M followers,NaN,image,NaN,1m


In [170]:
unique_dates = df['date_difference'].unique()
print(unique_dates)

['1m']


In [171]:

# Function to calculate post date
def get_post_date(relative_time, scrape_date):
    match = re.match(r"(\d+)(m|h|d|y)", relative_time)
    if match:
        value, unit = int(match.group(1)), match.group(2)
        if unit == "m":
            return scrape_date - timedelta(minutes=value)
        elif unit == "h":
            return scrape_date - timedelta(hours=value)
        elif unit == "f":
            return scrape_date - timedelta(days=value)
        elif unit == "y":
            return scrape_date.replace(year=scrape_date.year - value)

    return None

In [172]:
# Apply function to DataFrame
df['date_scraped'] = pd.to_datetime(df['date_scraped'])
df['date'] = df.apply(lambda row: get_post_date(row['date_difference'], row['date_scraped']), axis=1)
# Print results
df


,user_id,platform,post_id,date,likes,comments,shares,post_text,post_origin_text,date_scraped,views,followers,country,content_type,sentiment_score,date_difference
0,aljazeerachannel,Facebook,27c8d322-f361-48cc-bf5b-a886062806f0,2025-03-09 22:17:20,356,64,12,Breaking | Reuters on diplomats: #Washington a...,Al Jazeera Channel - قناة الجزيرة\n n\nd\no\ne...,2025-03-09 22:18:20,NaN,30M followers,NaN,image,NaN,1m


In [173]:
unique_likes = df['likes'].unique()
print(unique_likes)

[356]


In [158]:
# Function to convert likes count to integer
def parse_likes_comments_shares_views_followers(values):
    values = str(values).replace(" ", "")  # Remove all spaces   
    match = re.match(r"(\d+(\.\d+)?)([KM]?)", values)
    if match:
        value, suffix = float(match.group(1)), match.group(3)
        if suffix == "K":
            return int(value * 1_000)
        elif suffix == "M":
            return int(value * 1_000_000)
        return int(value)
    return 0

In [159]:
# Apply the function to the 'Likes' column
df['likes'] = df['likes'].apply(parse_likes_comments_shares_views_followers)
df

,user_id,platform,post_id,date,likes,comments,shares,views,followers,country,content_type,sentiment_score,post_text,post_origin_text,date_scraped,date_difference
0,aljazeerachannel,Facebook,c83048a0-3b7e-4ac1-9dd8-35be5a6ecfab,2025-03-06 17:45:07,406,55,18,NaN,20 M,NaN,video,NaN,"A Palestinian mother's shock, the moment she r...",Al Jazeera Channel - قناة الجزيرة \nS\nd\ne\nn...,2025-03-06 18:39:07,54m
1,aljazeerachannel,Facebook,69e34b53-c500-4eff-8d4a-f3c208b4b79b,2025-03-06 17:39:38,5900,927,87,NaN,20 M,NaN,video,NaN,Live on the air.. Violent clashes between the ...,Al Jazeera Channel - قناة الجزيرة \no\ns\nr\nS...,2025-03-06 18:39:38,1h
2,aljazeerachannel,Facebook,03d3f92a-da56-4320-bd08-a64f3cf38c0e,2025-03-06 17:40:02,151,NaN,NaN,NaN,NaN,NaN,video,NaN,#FIFA announces a unique offer in the final #F...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:40:02,1h
3,aljazeerachannel,Facebook,6f000c7c-47eb-4349-bfa8-4a17e74f8404,2025-03-06 17:45:15,767,28,48,NaN,NaN,NaN,video,NaN,“With Palestinian and decorations”.. Children ...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:40:15,55m
4,aljazeerachannel,Facebook,9d32629d-adc1-4bb5-937e-7797a1bbe47e,2025-03-06 17:52:43,317,21,3,NaN,NaN,NaN,video,NaN,Is there a common Qasim between Netanyahu and ...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:41:43,49m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,aljazeerachannel,Facebook,b182c81d-ab85-48fd-b728-3677d8d1e99b,2024-03-06 19:39:37,3500,33,53,NaN,NaN,NaN,video,NaN,مخاوف من عواقب “خطيرة وبعيدة المدى” إثر استهدا...,Al Jazeera Channel - قناة الجزيرة \np\nd\ns\nS...,2025-03-06 19:39:37,1y
254,aljazeerachannel,Facebook,76e04484-13c8-4cb8-a114-1b4befc0bc9e,2024-03-06 19:39:47,220,8,6,NaN,NaN,NaN,image,NaN,حرب العملات الرقمية المشفرة بين #حماس وإسرائيل,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:39:47,1y
255,aljazeerachannel,Facebook,52a203ef-40ca-440c-b916-da85e6775c44,2024-03-06 19:39:57,230,11,6,NaN,NaN,NaN,video,NaN,تفاعل على منصات التواصل مع كشف شركة “آبل” عن ن...,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:39:57,1y
256,aljazeerachannel,Facebook,be830853-40df-4b05-b272-883bdd578dff,2024-03-06 19:40:08,259,15,4,NaN,NaN,NaN,video,NaN,الاتحاد الدولي للنقل الجوي يتوقع عودة شركات ال...,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:40:08,1y


In [160]:
df['comments'] = df['comments'].fillna(0)

df['comments'] = df['comments'].apply(parse_likes_comments_shares_views_followers)

df

,user_id,platform,post_id,date,likes,comments,shares,views,followers,country,content_type,sentiment_score,post_text,post_origin_text,date_scraped,date_difference
0,aljazeerachannel,Facebook,c83048a0-3b7e-4ac1-9dd8-35be5a6ecfab,2025-03-06 17:45:07,406,55,18,NaN,20 M,NaN,video,NaN,"A Palestinian mother's shock, the moment she r...",Al Jazeera Channel - قناة الجزيرة \nS\nd\ne\nn...,2025-03-06 18:39:07,54m
1,aljazeerachannel,Facebook,69e34b53-c500-4eff-8d4a-f3c208b4b79b,2025-03-06 17:39:38,5900,927,87,NaN,20 M,NaN,video,NaN,Live on the air.. Violent clashes between the ...,Al Jazeera Channel - قناة الجزيرة \no\ns\nr\nS...,2025-03-06 18:39:38,1h
2,aljazeerachannel,Facebook,03d3f92a-da56-4320-bd08-a64f3cf38c0e,2025-03-06 17:40:02,151,0,NaN,NaN,NaN,NaN,video,NaN,#FIFA announces a unique offer in the final #F...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:40:02,1h
3,aljazeerachannel,Facebook,6f000c7c-47eb-4349-bfa8-4a17e74f8404,2025-03-06 17:45:15,767,28,48,NaN,NaN,NaN,video,NaN,“With Palestinian and decorations”.. Children ...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:40:15,55m
4,aljazeerachannel,Facebook,9d32629d-adc1-4bb5-937e-7797a1bbe47e,2025-03-06 17:52:43,317,21,3,NaN,NaN,NaN,video,NaN,Is there a common Qasim between Netanyahu and ...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:41:43,49m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,aljazeerachannel,Facebook,b182c81d-ab85-48fd-b728-3677d8d1e99b,2024-03-06 19:39:37,3500,33,53,NaN,NaN,NaN,video,NaN,مخاوف من عواقب “خطيرة وبعيدة المدى” إثر استهدا...,Al Jazeera Channel - قناة الجزيرة \np\nd\ns\nS...,2025-03-06 19:39:37,1y
254,aljazeerachannel,Facebook,76e04484-13c8-4cb8-a114-1b4befc0bc9e,2024-03-06 19:39:47,220,8,6,NaN,NaN,NaN,image,NaN,حرب العملات الرقمية المشفرة بين #حماس وإسرائيل,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:39:47,1y
255,aljazeerachannel,Facebook,52a203ef-40ca-440c-b916-da85e6775c44,2024-03-06 19:39:57,230,11,6,NaN,NaN,NaN,video,NaN,تفاعل على منصات التواصل مع كشف شركة “آبل” عن ن...,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:39:57,1y
256,aljazeerachannel,Facebook,be830853-40df-4b05-b272-883bdd578dff,2024-03-06 19:40:08,259,15,4,NaN,NaN,NaN,video,NaN,الاتحاد الدولي للنقل الجوي يتوقع عودة شركات ال...,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:40:08,1y


In [161]:
df['shares'] = df['shares'].fillna(0)

df['shares'] = df['shares'].apply(parse_likes_comments_shares_views_followers)

df

,user_id,platform,post_id,date,likes,comments,shares,views,followers,country,content_type,sentiment_score,post_text,post_origin_text,date_scraped,date_difference
0,aljazeerachannel,Facebook,c83048a0-3b7e-4ac1-9dd8-35be5a6ecfab,2025-03-06 17:45:07,406,55,18,NaN,20 M,NaN,video,NaN,"A Palestinian mother's shock, the moment she r...",Al Jazeera Channel - قناة الجزيرة \nS\nd\ne\nn...,2025-03-06 18:39:07,54m
1,aljazeerachannel,Facebook,69e34b53-c500-4eff-8d4a-f3c208b4b79b,2025-03-06 17:39:38,5900,927,87,NaN,20 M,NaN,video,NaN,Live on the air.. Violent clashes between the ...,Al Jazeera Channel - قناة الجزيرة \no\ns\nr\nS...,2025-03-06 18:39:38,1h
2,aljazeerachannel,Facebook,03d3f92a-da56-4320-bd08-a64f3cf38c0e,2025-03-06 17:40:02,151,0,0,NaN,NaN,NaN,video,NaN,#FIFA announces a unique offer in the final #F...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:40:02,1h
3,aljazeerachannel,Facebook,6f000c7c-47eb-4349-bfa8-4a17e74f8404,2025-03-06 17:45:15,767,28,48,NaN,NaN,NaN,video,NaN,“With Palestinian and decorations”.. Children ...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:40:15,55m
4,aljazeerachannel,Facebook,9d32629d-adc1-4bb5-937e-7797a1bbe47e,2025-03-06 17:52:43,317,21,3,NaN,NaN,NaN,video,NaN,Is there a common Qasim between Netanyahu and ...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:41:43,49m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,aljazeerachannel,Facebook,b182c81d-ab85-48fd-b728-3677d8d1e99b,2024-03-06 19:39:37,3500,33,53,NaN,NaN,NaN,video,NaN,مخاوف من عواقب “خطيرة وبعيدة المدى” إثر استهدا...,Al Jazeera Channel - قناة الجزيرة \np\nd\ns\nS...,2025-03-06 19:39:37,1y
254,aljazeerachannel,Facebook,76e04484-13c8-4cb8-a114-1b4befc0bc9e,2024-03-06 19:39:47,220,8,6,NaN,NaN,NaN,image,NaN,حرب العملات الرقمية المشفرة بين #حماس وإسرائيل,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:39:47,1y
255,aljazeerachannel,Facebook,52a203ef-40ca-440c-b916-da85e6775c44,2024-03-06 19:39:57,230,11,6,NaN,NaN,NaN,video,NaN,تفاعل على منصات التواصل مع كشف شركة “آبل” عن ن...,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:39:57,1y
256,aljazeerachannel,Facebook,be830853-40df-4b05-b272-883bdd578dff,2024-03-06 19:40:08,259,15,4,NaN,NaN,NaN,video,NaN,الاتحاد الدولي للنقل الجوي يتوقع عودة شركات ال...,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:40:08,1y


In [162]:
df['views'] = df['views'].fillna(0)

df['views'] = df['views'].apply(parse_likes_comments_shares_views_followers)

df

,user_id,platform,post_id,date,likes,comments,shares,views,followers,country,content_type,sentiment_score,post_text,post_origin_text,date_scraped,date_difference
0,aljazeerachannel,Facebook,c83048a0-3b7e-4ac1-9dd8-35be5a6ecfab,2025-03-06 17:45:07,406,55,18,0,20 M,NaN,video,NaN,"A Palestinian mother's shock, the moment she r...",Al Jazeera Channel - قناة الجزيرة \nS\nd\ne\nn...,2025-03-06 18:39:07,54m
1,aljazeerachannel,Facebook,69e34b53-c500-4eff-8d4a-f3c208b4b79b,2025-03-06 17:39:38,5900,927,87,0,20 M,NaN,video,NaN,Live on the air.. Violent clashes between the ...,Al Jazeera Channel - قناة الجزيرة \no\ns\nr\nS...,2025-03-06 18:39:38,1h
2,aljazeerachannel,Facebook,03d3f92a-da56-4320-bd08-a64f3cf38c0e,2025-03-06 17:40:02,151,0,0,0,NaN,NaN,video,NaN,#FIFA announces a unique offer in the final #F...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:40:02,1h
3,aljazeerachannel,Facebook,6f000c7c-47eb-4349-bfa8-4a17e74f8404,2025-03-06 17:45:15,767,28,48,0,NaN,NaN,video,NaN,“With Palestinian and decorations”.. Children ...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:40:15,55m
4,aljazeerachannel,Facebook,9d32629d-adc1-4bb5-937e-7797a1bbe47e,2025-03-06 17:52:43,317,21,3,0,NaN,NaN,video,NaN,Is there a common Qasim between Netanyahu and ...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:41:43,49m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,aljazeerachannel,Facebook,b182c81d-ab85-48fd-b728-3677d8d1e99b,2024-03-06 19:39:37,3500,33,53,0,NaN,NaN,video,NaN,مخاوف من عواقب “خطيرة وبعيدة المدى” إثر استهدا...,Al Jazeera Channel - قناة الجزيرة \np\nd\ns\nS...,2025-03-06 19:39:37,1y
254,aljazeerachannel,Facebook,76e04484-13c8-4cb8-a114-1b4befc0bc9e,2024-03-06 19:39:47,220,8,6,0,NaN,NaN,image,NaN,حرب العملات الرقمية المشفرة بين #حماس وإسرائيل,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:39:47,1y
255,aljazeerachannel,Facebook,52a203ef-40ca-440c-b916-da85e6775c44,2024-03-06 19:39:57,230,11,6,0,NaN,NaN,video,NaN,تفاعل على منصات التواصل مع كشف شركة “آبل” عن ن...,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:39:57,1y
256,aljazeerachannel,Facebook,be830853-40df-4b05-b272-883bdd578dff,2024-03-06 19:40:08,259,15,4,0,NaN,NaN,video,NaN,الاتحاد الدولي للنقل الجوي يتوقع عودة شركات ال...,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:40:08,1y


In [163]:
df['followers'] = df['followers'].fillna(0)

df['followers'] = df['followers'].apply(parse_likes_comments_shares_views_followers)

df

,user_id,platform,post_id,date,likes,comments,shares,views,followers,country,content_type,sentiment_score,post_text,post_origin_text,date_scraped,date_difference
0,aljazeerachannel,Facebook,c83048a0-3b7e-4ac1-9dd8-35be5a6ecfab,2025-03-06 17:45:07,406,55,18,0,20000000,NaN,video,NaN,"A Palestinian mother's shock, the moment she r...",Al Jazeera Channel - قناة الجزيرة \nS\nd\ne\nn...,2025-03-06 18:39:07,54m
1,aljazeerachannel,Facebook,69e34b53-c500-4eff-8d4a-f3c208b4b79b,2025-03-06 17:39:38,5900,927,87,0,20000000,NaN,video,NaN,Live on the air.. Violent clashes between the ...,Al Jazeera Channel - قناة الجزيرة \no\ns\nr\nS...,2025-03-06 18:39:38,1h
2,aljazeerachannel,Facebook,03d3f92a-da56-4320-bd08-a64f3cf38c0e,2025-03-06 17:40:02,151,0,0,0,0,NaN,video,NaN,#FIFA announces a unique offer in the final #F...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:40:02,1h
3,aljazeerachannel,Facebook,6f000c7c-47eb-4349-bfa8-4a17e74f8404,2025-03-06 17:45:15,767,28,48,0,0,NaN,video,NaN,“With Palestinian and decorations”.. Children ...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:40:15,55m
4,aljazeerachannel,Facebook,9d32629d-adc1-4bb5-937e-7797a1bbe47e,2025-03-06 17:52:43,317,21,3,0,0,NaN,video,NaN,Is there a common Qasim between Netanyahu and ...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:41:43,49m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,aljazeerachannel,Facebook,b182c81d-ab85-48fd-b728-3677d8d1e99b,2024-03-06 19:39:37,3500,33,53,0,0,NaN,video,NaN,مخاوف من عواقب “خطيرة وبعيدة المدى” إثر استهدا...,Al Jazeera Channel - قناة الجزيرة \np\nd\ns\nS...,2025-03-06 19:39:37,1y
254,aljazeerachannel,Facebook,76e04484-13c8-4cb8-a114-1b4befc0bc9e,2024-03-06 19:39:47,220,8,6,0,0,NaN,image,NaN,حرب العملات الرقمية المشفرة بين #حماس وإسرائيل,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:39:47,1y
255,aljazeerachannel,Facebook,52a203ef-40ca-440c-b916-da85e6775c44,2024-03-06 19:39:57,230,11,6,0,0,NaN,video,NaN,تفاعل على منصات التواصل مع كشف شركة “آبل” عن ن...,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:39:57,1y
256,aljazeerachannel,Facebook,be830853-40df-4b05-b272-883bdd578dff,2024-03-06 19:40:08,259,15,4,0,0,NaN,video,NaN,الاتحاد الدولي للنقل الجوي يتوقع عودة شركات ال...,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:40:08,1y


In [164]:
df['country'] = df['country'].fillna("")
df

,user_id,platform,post_id,date,likes,comments,shares,views,followers,country,content_type,sentiment_score,post_text,post_origin_text,date_scraped,date_difference
0,aljazeerachannel,Facebook,c83048a0-3b7e-4ac1-9dd8-35be5a6ecfab,2025-03-06 17:45:07,406,55,18,0,20000000,,video,NaN,"A Palestinian mother's shock, the moment she r...",Al Jazeera Channel - قناة الجزيرة \nS\nd\ne\nn...,2025-03-06 18:39:07,54m
1,aljazeerachannel,Facebook,69e34b53-c500-4eff-8d4a-f3c208b4b79b,2025-03-06 17:39:38,5900,927,87,0,20000000,,video,NaN,Live on the air.. Violent clashes between the ...,Al Jazeera Channel - قناة الجزيرة \no\ns\nr\nS...,2025-03-06 18:39:38,1h
2,aljazeerachannel,Facebook,03d3f92a-da56-4320-bd08-a64f3cf38c0e,2025-03-06 17:40:02,151,0,0,0,0,,video,NaN,#FIFA announces a unique offer in the final #F...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:40:02,1h
3,aljazeerachannel,Facebook,6f000c7c-47eb-4349-bfa8-4a17e74f8404,2025-03-06 17:45:15,767,28,48,0,0,,video,NaN,“With Palestinian and decorations”.. Children ...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:40:15,55m
4,aljazeerachannel,Facebook,9d32629d-adc1-4bb5-937e-7797a1bbe47e,2025-03-06 17:52:43,317,21,3,0,0,,video,NaN,Is there a common Qasim between Netanyahu and ...,Al Jazeera Channel - قناة الجزيرة \nr\nn\ns\ne...,2025-03-06 18:41:43,49m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,aljazeerachannel,Facebook,b182c81d-ab85-48fd-b728-3677d8d1e99b,2024-03-06 19:39:37,3500,33,53,0,0,,video,NaN,مخاوف من عواقب “خطيرة وبعيدة المدى” إثر استهدا...,Al Jazeera Channel - قناة الجزيرة \np\nd\ns\nS...,2025-03-06 19:39:37,1y
254,aljazeerachannel,Facebook,76e04484-13c8-4cb8-a114-1b4befc0bc9e,2024-03-06 19:39:47,220,8,6,0,0,,image,NaN,حرب العملات الرقمية المشفرة بين #حماس وإسرائيل,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:39:47,1y
255,aljazeerachannel,Facebook,52a203ef-40ca-440c-b916-da85e6775c44,2024-03-06 19:39:57,230,11,6,0,0,,video,NaN,تفاعل على منصات التواصل مع كشف شركة “آبل” عن ن...,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:39:57,1y
256,aljazeerachannel,Facebook,be830853-40df-4b05-b272-883bdd578dff,2024-03-06 19:40:08,259,15,4,0,0,,video,NaN,الاتحاد الدولي للنقل الجوي يتوقع عودة شركات ال...,Al Jazeera Channel - قناة الجزيرة \nn\np\ne\no...,2025-03-06 19:40:08,1y


In [183]:
df_post_youtube = pd.read_excel('../Scraping_Output/youtube_data.xlsx')
df_post_youtube

,user_id,platform,post_id,date,likes,comments,shares,post_text,post_origin_text,date_scraped,views,followers,country,content_type,sentiment_score
0,aljazeerachannel,Facebook,27c8d322-f361-48cc-bf5b-a886062806f0,NaN,356,64,12.0,Breaking | Reuters on diplomats: #Washington a...,Al Jazeera Channel - قناة الجزيرة\n n\nd\no\ne...,2025-03-09 22:18:20,NaN,30M followers,NaN,image,NaN
1,aljazeera,Youtube,c675acab-962e-4baa-ae5e-93f435b9c11f,1 hour ago,591,26,NaN,رئيس البرلمان الإيراني: تصرفات ترمب مع الدول ا...,AlJazeera Arabic قناة الجزيرة\n1 hour ago\nرئي...,2025-03-09 22:18:45,NaN,19.9M subscribers,NaN,image,NaN
2,aljazeera,Youtube,ac9a0faf-21c0-434b-9a04-51ee76f389c2,1 hour ago,2.3K,138,NaN,#عاجل | الرئيس السوري: المخاطر التي نواجهها هي...,AlJazeera Arabic قناة الجزيرة\n1 hour ago\n#عا...,2025-03-09 22:18:50,NaN,19.9M subscribers,NaN,image,NaN
3,aljazeera,Youtube,b7664017-dd3f-4a36-89cd-8314b40341f0,3 hours ago,531,40,NaN,#إيلون_ماسك: نظام ستارلينك هو العمود الفقري لل...,AlJazeera Arabic قناة الجزيرة\n3 hours ago\n#إ...,2025-03-09 22:18:55,NaN,19.9M subscribers,NaN,image,NaN
4,aljazeera,Youtube,6eb73b86-3a30-4178-a752-ce73b761a568,5 hours ago,6.7K,125,NaN,#حماس: وافقنا على تشكيل لجنة من شخصيات وطنية م...,AlJazeera Arabic قناة الجزيرة\n5 hours ago\n#ح...,2025-03-09 22:19:00,NaN,19.9M subscribers,NaN,image,NaN
5,aljazeera,Youtube,f8aa20c0-009f-4e1a-ac68-b452ac8e6c02,6 hours ago,994,42,NaN,#عاجل | مصادر للجزيرة: دبابات الاحتلال تقتحم م...,AlJazeera Arabic قناة الجزيرة\n6 hours ago\n#ع...,2025-03-09 22:19:05,NaN,19.9M subscribers,NaN,image,NaN
6,aljazeera,Youtube,b9784d68-ecac-4573-abcb-90b597722957,7 hours ago,318,65,NaN,"نتنياهو يشكر #ترمب على دعمه ""الجريء"" لإسرائيل ...",AlJazeera Arabic قناة الجزيرة\n7 hours ago\nنت...,2025-03-09 22:19:11,NaN,19.9M subscribers,NaN,image,NaN
7,aljazeera,Youtube,8e605043-4890-4946-add3-7f0b5493b85d,7 hours ago,986,109,NaN,#عاجل | مصدر أمني للجزيرة: الانتهاء من تمشيط ب...,AlJazeera Arabic قناة الجزيرة\n7 hours ago\n#ع...,2025-03-09 22:19:17,NaN,19.9M subscribers,NaN,image,NaN
8,aljazeera,Youtube,fa242f96-ac87-46c7-bfbb-602e72b74f16,8 hours ago,912,57,NaN,#حدث_في_رمضان | على مقربة من سهل بطليوس بإسبان...,AlJazeera Arabic قناة الجزيرة\n8 hours ago\n#ح...,2025-03-09 22:19:22,NaN,19.9M subscribers,NaN,image,NaN
9,aljazeera,Youtube,76988744-a554-43b3-b4be-6c2a3dbae65b,9 hours ago,524,59,NaN,خامنئي ردا على دعوة #ترمب للتفاوض: إصرار بعض ا...,AlJazeera Arabic قناة الجزيرة\n9 hours ago\nخا...,2025-03-09 22:19:28,NaN,19.9M subscribers,NaN,image,NaN


In [186]:
    # Function to calculate post date
    def get_youtube_post_date(relative_time, scrape_date):
        match = re.match(r"(\d+) (minute|hour|day|year)s? ago", relative_time)
        if match:
            value, unit = int(match.group(1)), match.group(2)
            if unit == "minute":
                return scrape_date - timedelta(minutes=value)
            elif unit == "hour":
                return scrape_date - timedelta(hours=value)
            elif unit == "day":
                return scrape_date - timedelta(days=value)
            elif unit == "year":
                return scrape_date.replace(year=scrape_date.year - value)
        elif "edited" in relative_time:
            return get_post_date(relative_time.replace(" (edited)", ""))
        return None

In [188]:
        # Remove rows where 'post_origin_text' is empty (i.e., NaN)
        df_post_youtube = df_post_youtube.dropna(subset=['post_origin_text'])

        # Reset the index of the DataFrame after dropping rows, without keeping the old index
        df_post_youtube = df_post_youtube.reset_index(drop=True)
                
        # Convert the 'date_scraped' column to datetime format
        df_post_youtube['date_scraped'] = pd.to_datetime(df_post_youtube['date_scraped'])
        df_post_youtube
        # # Apply a custom function to calculate 'date' based on 'date_scraped' for each row
        # df_post_youtube['date'] = df_post_youtube.apply(lambda row: get_youtube_post_date(row['date'], row['date_scraped']), axis=1)

,user_id,platform,post_id,date,likes,comments,shares,post_text,post_origin_text,date_scraped,views,followers,country,content_type,sentiment_score
0,aljazeerachannel,Facebook,27c8d322-f361-48cc-bf5b-a886062806f0,NaN,356,64,12.0,Breaking | Reuters on diplomats: #Washington a...,Al Jazeera Channel - قناة الجزيرة\n n\nd\no\ne...,2025-03-09 22:18:20,NaN,30M followers,NaN,image,NaN
1,aljazeera,Youtube,c675acab-962e-4baa-ae5e-93f435b9c11f,1 hour ago,591,26,NaN,رئيس البرلمان الإيراني: تصرفات ترمب مع الدول ا...,AlJazeera Arabic قناة الجزيرة\n1 hour ago\nرئي...,2025-03-09 22:18:45,NaN,19.9M subscribers,NaN,image,NaN
2,aljazeera,Youtube,ac9a0faf-21c0-434b-9a04-51ee76f389c2,1 hour ago,2.3K,138,NaN,#عاجل | الرئيس السوري: المخاطر التي نواجهها هي...,AlJazeera Arabic قناة الجزيرة\n1 hour ago\n#عا...,2025-03-09 22:18:50,NaN,19.9M subscribers,NaN,image,NaN
3,aljazeera,Youtube,b7664017-dd3f-4a36-89cd-8314b40341f0,3 hours ago,531,40,NaN,#إيلون_ماسك: نظام ستارلينك هو العمود الفقري لل...,AlJazeera Arabic قناة الجزيرة\n3 hours ago\n#إ...,2025-03-09 22:18:55,NaN,19.9M subscribers,NaN,image,NaN
4,aljazeera,Youtube,6eb73b86-3a30-4178-a752-ce73b761a568,5 hours ago,6.7K,125,NaN,#حماس: وافقنا على تشكيل لجنة من شخصيات وطنية م...,AlJazeera Arabic قناة الجزيرة\n5 hours ago\n#ح...,2025-03-09 22:19:00,NaN,19.9M subscribers,NaN,image,NaN
5,aljazeera,Youtube,f8aa20c0-009f-4e1a-ac68-b452ac8e6c02,6 hours ago,994,42,NaN,#عاجل | مصادر للجزيرة: دبابات الاحتلال تقتحم م...,AlJazeera Arabic قناة الجزيرة\n6 hours ago\n#ع...,2025-03-09 22:19:05,NaN,19.9M subscribers,NaN,image,NaN
6,aljazeera,Youtube,b9784d68-ecac-4573-abcb-90b597722957,7 hours ago,318,65,NaN,"نتنياهو يشكر #ترمب على دعمه ""الجريء"" لإسرائيل ...",AlJazeera Arabic قناة الجزيرة\n7 hours ago\nنت...,2025-03-09 22:19:11,NaN,19.9M subscribers,NaN,image,NaN
7,aljazeera,Youtube,8e605043-4890-4946-add3-7f0b5493b85d,7 hours ago,986,109,NaN,#عاجل | مصدر أمني للجزيرة: الانتهاء من تمشيط ب...,AlJazeera Arabic قناة الجزيرة\n7 hours ago\n#ع...,2025-03-09 22:19:17,NaN,19.9M subscribers,NaN,image,NaN
8,aljazeera,Youtube,fa242f96-ac87-46c7-bfbb-602e72b74f16,8 hours ago,912,57,NaN,#حدث_في_رمضان | على مقربة من سهل بطليوس بإسبان...,AlJazeera Arabic قناة الجزيرة\n8 hours ago\n#ح...,2025-03-09 22:19:22,NaN,19.9M subscribers,NaN,image,NaN
9,aljazeera,Youtube,76988744-a554-43b3-b4be-6c2a3dbae65b,9 hours ago,524,59,NaN,خامنئي ردا على دعوة #ترمب للتفاوض: إصرار بعض ا...,AlJazeera Arabic قناة الجزيرة\n9 hours ago\nخا...,2025-03-09 22:19:28,NaN,19.9M subscribers,NaN,image,NaN
